In [ ]:
# Lib to work with dataframes
import pandas as pd
# Command to not show some warnings
pd.options.mode.chained_assignment = None
# Lib to do calculations
import numpy as np

In [ ]:
# Read the streamings's datasets
df_prime = pd.read_csv('df_prime.csv', sep=';')
df_disney_plus = pd.read_csv('df_disney_plus.csv', sep=';')
df_hbo_max = pd.read_csv('df_hbo_max.csv', sep=';')
df_hulu = pd.read_csv('df_hulu.csv', sep=';')
df_netflix = pd.read_csv('df_netflix.csv', sep=';')

In [ ]:
# Creat a new column in each dataset to identify each one when merging then
df_netflix['streaming'] = 'netflix'
df_prime['streaming'] = 'prime'
df_disney_plus['streaming'] = 'disney +'
df_hulu['streaming'] = 'hulu'
df_hbo_max['streaming'] = 'hbo max'

In [ ]:
# Merge the datasets
df = pd.concat([df_netflix, df_prime, df_disney_plus, df_hulu, df_hbo_max])

In [ ]:
# Creating a new index for the entire merged dataset and dropping the old one
df.reset_index(inplace=True)
df.drop(columns='index', inplace=True)

In [ ]:
# Read the imdb's datasets
titles_imdb = pd.read_csv('title.basics.tsv.gz', compression='gzip', sep='\t')
ranking_imdb = pd.read_csv('title.ratings.tsv.gz', compression='gzip', sep='\t')

### Look the datas

In [ ]:
df.info()

In [ ]:
titles_imdb.info()

In [ ]:
ranking_imdb.info()

### Searching and cleaning the unnecessary datas

In [ ]:
# Dropping titles with too small number of votes
ranking_imdb.drop(ranking_imdb[ranking_imdb.numVotes < 2000].index, inplace=True)

In [ ]:
titles_imdb.titleType.unique()

In [ ]:
titles_imdb = titles_imdb[titles_imdb.titleType != 'videoGame']

In [ ]:
titles_imdb.startYear.unique()

In [ ]:
titles_imdb.drop(titles_imdb[titles_imdb.startYear == '\\N'].index, inplace=True)

In [ ]:
titles_imdb.startYear = titles_imdb.startYear.astype('int')

### Draw attention the large number of titles with null values in the streaming dataset

#### To handle with this null values, the best way is get the values from the imdb's dataset. However, these datasets are very large and this process can take a long time. To reduce the time it is good to divide the datasets by a common attribute, in this case I choose the year.

In [ ]:
df.rename(columns={'release_year' : 'startYear'}, inplace=True)

In [ ]:
df.info()

In [ ]:
df.startYear.sort_values()

In [ ]:
first_year = 1920
first_qrt = df.startYear.quantile(q=.25)
second_qrt = df.startYear.quantile()
third_qrt = df.startYear.quantile(q=.75)

In [ ]:
def split_dataset(dataset):
    dataset_first_qrt = dataset[(dataset.startYear >= first_year) & (dataset.startYear < first_qrt)]
    dataset_second_qrt = dataset[(dataset.startYear >= first_qrt) & (dataset.startYear < second_qrt)]
    dataset_third_qrt = dataset[(dataset.startYear >= second_qrt) & (dataset.startYear < third_qrt)]
    dataset_last_qrt = dataset[dataset.startYear >= third_qrt]
    return dataset_first_qrt, dataset_second_qrt, dataset_third_qrt, dataset_last_qrt

In [ ]:
df_first_qrt, df_second_qrt, df_third_qrt, df_last_qrt = split_dataset(df)

In [ ]:
titles_imdb_first_qrt, titles_imdb_second_qrt, titles_imdb_third_qrt, titles_imdb_last_qrt = split_dataset(titles_imdb)

In [ ]:
def get_tconst(df_dataset, titles_imdb_dataset):
    tconst = []
    n = 1
    for i in df_dataset.index:
        print(round(n/len(df_dataset) * 100, 2), '%')
        n += 1
        title = df_dataset.title[i]
        year = df_dataset.startYear[i]
        id_imdb = titles_imdb_dataset.query('originalTitle == @title | primaryTitle == @title & startYear == @year')['tconst'].values
        for j in range(len(id_imdb)):
            if id_imdb[j] in ranking_imdb.tconst.values:
                tconst.append(id_imdb[j])
    return tconst

In [ ]:
tconst_first_qrt = get_tconst(df_first_qrt, titles_imdb_first_qrt)

In [ ]:
tconst_second_qrt = get_tconst(df_second_qrt, titles_imdb_second_qrt)

In [ ]:
tconst_third_qrt = get_tconst(df_third_qrt, titles_imdb_third_qrt)

In [ ]:
tconst_last_qrt = get_tconst(df_last_qrt, titles_imdb_last_qrt)

In [ ]:
def add_score(tconst_list, titles_imdb_qrt):
    n = 1
    titles_imdb_qrt['averageRating'] = 0
    for tconst in tconst_list:
        print(round(n/len(tconst_list) * 100, 2), '%')
        n += 1
        id_title = titles_imdb_qrt[titles_imdb_qrt.tconst == tconst].index[0] 
        id_score = ranking_imdb[ranking_imdb.tconst == tconst].index[0]
        titles_imdb_qrt['averageRating'][id_title] = ranking_imdb['averageRating'][id_score]
    return titles_imdb_qrt

In [ ]:
titles_imdb_first_qrt = add_score(tconst_first_qrt, titles_imdb_first_qrt)

In [ ]:
titles_imdb_second_qrt = add_score(tconst_second_qrt, titles_imdb_second_qrt)

In [ ]:
titles_imdb_third_qrt = add_score(tconst_third_qrt, titles_imdb_third_qrt)

In [ ]:
titles_imdb_last_qrt = add_score(tconst_last_qrt, titles_imdb_last_qrt)

In [ ]:
titles_imdb_first_qrt = titles_imdb_first_qrt[titles_imdb_first_qrt.averageRating != 0]
titles_imdb_first_qrt

In [ ]:
titles_imdb_second_qrt = titles_imdb_second_qrt[titles_imdb_second_qrt.averageRating != 0]
titles_imdb_second_qrt

In [ ]:
titles_imdb_third_qrt = titles_imdb_third_qrt[titles_imdb_third_qrt.averageRating != 0]
titles_imdb_third_qrt

In [ ]:
titles_imdb_last_qrt = titles_imdb_last_qrt[titles_imdb_last_qrt.averageRating != 0]
titles_imdb_last_qrt

In [ ]:
titles_imdb_with_scores = pd.concat([titles_imdb_first_qrt, titles_imdb_second_qrt, titles_imdb_third_qrt, titles_imdb_last_qrt])

In [ ]:
titles_imdb_with_scores.to_csv('titles_imdb_with_scores.csv', sep=';', index = False, encoding = 'utf-8-sig')

In [ ]:
titles_imdb_with_scores = pd.read_csv('titles_imdb_with_scores.csv', sep=';')
titles_imdb_with_scores

In [ ]:
titles_imdb_with_scores[titles_imdb_with_scores.duplicated(subset=['originalTitle', 'startYear'], keep=False)].sort_values(by='originalTitle')

In [ ]:
titles_imdb_with_scores[titles_imdb_with_scores.duplicated(subset='originalTitle', keep=False)].sort_values(by='originalTitle')

In [ ]:
n = 1
for i in df[df.average_rating.isna()].index:
    print(round(n/len(df) * 100, 2), '%')
    n += 1
    title = df.title[i]
    year = df.startYear[i]
    id_imdb = titles_imdb_with_scores.query('originalTitle == @title | primaryTitle == @title & startYear == @year')['tconst'].values
    if len(id_imdb) > 1:
        tconst_amb = []
        for j in range(len(id_imdb)):
            tconst_amb.append({title: id_imdb[j]})
    if len(id_imdb) == 1:
        id_score = titles_imdb_with_scores[titles_imdb_with_scores.tconst == id_imdb[0]].index[0]
        df.average_rating[i] = titles_imdb_with_scores.averageRating[id_score]

In [ ]:
tconst_amb

In [ ]:
df[df.title == 'Witness']

In [ ]:
titles_imdb_with_scores[titles_imdb_with_scores.originalTitle == 'Witness']

In [ ]:
df.dropna(subset='average_rating', inplace=True)

In [ ]:
df = df[df.average_rating != ' ']

In [ ]:
df.average_rating = df.average_rating.astype('float')
df.average_rating

In [ ]:
df.title = df.title.apply(lambda x: x.title())

In [ ]:
# Handling duplicate values
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df[df.duplicated(subset=['description', 'streaming'], keep=False)].sort_values(by='title')

In [ ]:
df.drop_duplicates(subset=['description', 'streaming'], inplace=True)

In [ ]:
# Resetting the index to avoid problems with algorithm results
df.reset_index(inplace=True)
df.drop(columns='index', inplace=True)

In [ ]:
# Noise in description
smalls_descrip = []
for i in df.index:
    if len(df.description[i]) < 20:
        smalls_descrip.append(i)
df.loc[smalls_descrip]

In [ ]:
df.description[5593] = 'A mob enforcers son in 1930s Illinois witnesses a murder, forcing him and his father to take to the road, and his father down a path of redemption and revenge.'

In [ ]:
df.description[10420] = 'An aspiring fashion designer is mysteriously able to enter the 1960s where she encounters a dazzling wannabe singer. But the glamour is not all it appears to be and the dreams of the past start to crack and splinter into something darker.'

In [ ]:
df.description[12263] = 'A young man is determined to solve an unspeakable crime and absolve the people he loves, while looking for answers within his fractured family and community.'

In [ ]:
# Development remains in the description
df[df.description.apply(lambda x: 'Test' in x)]

In [ ]:
df.info()

In [ ]:
df.genre = df.genre.apply(lambda x: str(x).strip('[]'))

In [ ]:
# Creating some new columns to help the prediction algorithm.
ratings_ages = {
    'TV-PG': 'Older Kids',
    'TV-MA': 'Adults',
    'TV-Y7-FV': 'Older Kids',
    'TV-Y7': 'Older Kids',
    'TV-14': 'Teens',
    'R': 'Adults',
    'TV-Y': 'Little Kids',
    'NR': 'Adults',
    'PG-13': 'Teens',
    'TV-G': 'Little Kids',
    'PG': 'Older Kids',
    'G': 'Little Kids',
    'UR': 'Adults',
    'NC-17': 'Adults',
    '7+' : 'Older Kids',
    '16+': 'Adults',
    '18+': 'Adults',
    '13+': 'Teens',
    'UNRATED': ' ',
    'NOT RATED': ' ',
    'ALL': 'ALL AGES',
    'ALL_AGES': 'AL AGES',
    'TV-NR': 'Adults',
    'AGES_18_': 'Adults',
}
df["parental_guidelines"]= df["parental_guidelines"].replace(ratings_ages)

In [ ]:
df['genre_and_description'] = df.genre + ', ' + df.description

In [ ]:
df.to_csv('df_prepared_recomendation_plus.csv', sep=';', index = False, encoding = 'utf-8-sig')